# Electoral Physics 

### Packages

In [17]:
# Data Management 
using DataFrames, DataFramesMeta
using DrWatson

# Statistics
using Random
using Distributions
using StatsBase

# Graphs 
using LightGraphs, SimpleWeightedGraphs, GraphIO
using GraphPlot
 #using Graphs

# Modelling
using Agents
include("./ElectoralPhysics.jl")

# Data Visualization
using Plots
using AgentsPlots
using PlotThemes

# Python
# ENV["PYTHON"] = "path/to/python"
# Pkg.build("PyCall")
using PyCall
using PyPlot
nx = pyimport("networkx")
np = pyimport("numpy")
nw = pyimport("netwulf");

### Agent Type 

In [4]:
# Voter Definition
mutable struct Voter <: AbstractAgent
    id::Int64             # ∈ ℕ
    pos::Int64            # ∈ ℕ
    candidacy::Bool       # ∈ 𝔹
    score::Float64        # ∈ 𝔹
    final_score::Float64  # \in \bbR
    community::Int64      # ∈ {1,2,3, ..., n}
end

### Micro-Dynamics

In [5]:
function vote!(agent,model)
    candidates = [agent for agent in allagents(model) if agent.candidacy == true]
    nvotes = nvotes!(agent, model)
    targets = preferential_attachment_choose_candidates!(agent, model, candidates, nvotes)
    for target in targets
        add_edge!(model.space.graph, agent.pos, target.pos, 1/nvotes)
    end
end

function nvotes!(agent, model)
    p = 0.1
    if agent.candidacy == true
        d = Binomial(model.properties[:M]-2,p)
        nvotes = rand(d)
    else 
        d = Binomial(model.properties[:M]-1,p)
        nvotes = rand(d)
    end
    
    return nvotes + 1
end

function choose_candidates!(agent, model, nvotes)
    candidates = [agent for agent in allagents(model) if agent.candidacy == true]
    targets = []
    for vote in 1:nvotes
        target = rand(candidates)
        while target.id == agent.id || target in targets
            target = rand(candidates)
        end
        
    push!(targets, target)
    end
    return targets
end


function preferential_attachment_choose_candidates!(agent, model, candidates, nvotes)
    agent_community = agent.community
    weights = Int64[]
    for candidate in candidates
        if candidate.community == agent_community
            push!(weights, 10)
        else
            push!(weights, 1)
        end
    end
    
    targets = []
    for vote in 1:nvotes
        target = StatsBase.sample(candidates, ProbabilityWeights(weights))
        while target.id == agent.id || target in targets
            target = StatsBase.sample(candidates, ProbabilityWeights(weights))
        end    
        push!(targets, target)
    end
    
    return targets
end

function update_score_indegree!(model)
    for agent in allagents(model)
        agent.score = agent.score + indegree_centrality(model.space.graph)[agent.id]
    end
end

function update_score_flow!(model)
    for agent in allagents(model)
        if agent.candidacy == true
            for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
                agent_inneighbor = get_node_agents(inneighbor, model)[1]
                if agent_inneighbor.candidacy == false
                    agent.score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
                end
            end
        end
    end
end

function update_score_pagerank!(model, α, n, ϵ)
    for agent in allagents(model)
        agent.score = pagerank(model.space.graph, α, n, ϵ)[agent.id]
    end
end

# Implement map equation !!

# IMPLEMENT NX TO LG

# Voter Dynamics 
function agent_step!(agent, model)
    vote!(agent, model)
end;

function update_score_candidates!(model)
    for agent in allagents(model)
        for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
            agent_inneighbor = get_node_agents(inneighbor, model)[1]
            agent.final_score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
        end
    end
end;

### Models

In [6]:
# Voter Model Initialization
function init_model_voters(N::Int64, M::Int64, K::Int64, community)
       
    properties = @dict(N, M, K)
    
    # Create Directed Weighted Graph
    space = GraphSpace(SimpleWeightedDiGraph(N)) 
    model_voters = ABM(Voter, space; properties = properties)
    
    
    for id in 1:N
        pos = id
        score = 1.0
        final_score = 0
        if id <= M
            candidacy = true
            add_agent!(pos, model_voters, candidacy, score, final_score, community[id])
        else
            candidacy = false
            add_agent!(pos, model_voters, candidacy, score, final_score, community[id])
        end
    end
    return model_voters
end;

# Candidate Model Initialization
function init_model_candidates(model_voters::AgentBasedModel{Voter,GraphSpace{SimpleWeightedDiGraph{Int64,Float64}},typeof(fastest),Dict{Symbol,Int64}}, 
                                M::Int64, K::Int64)
    properties = @dict(M, K)
    
    # Induce Subgraph
    graph, vertex_map = induced_subgraph(model_voters.space.graph, 1:M)
    space = GraphSpace(graph)
    model_candidates = ABM(Voter, space; properties = properties)
    
    k = 0
    for voter in allagents(model_voters)
        if voter.candidacy == true
            add_agent!(voter.pos, model_candidates, voter.candidacy, voter.score, voter.final_score, voter.community)
        end
    end
    
    for agent in allagents(model_candidates)
    agent.id = agent.pos
    end
    
    return model_candidates
end;

### Parameters

In [7]:
# Create population of voters 
N = 800

# Create subpopulation of candidates
M = 30;

# Create subpopulation of elected
K = 15;

# Initialize number of steps
nsteps = 1;


community = []
for i in 1:N
    if i <= M
        if i < M/3
            push!(community,1)
        elseif i >= M/3 && i < 2*(M/3)
            push!(community,2)
        else
            push!(community,3)
        end
    else
         if i < N/3
            push!(community,1)
        elseif i >= N/3 && i < 2*(N/3)
            push!(community,2)
        else
            push!(community,3)
        end
    end
end

α = 0.85
n = 500 
ϵ = 1e-6;

### Voter Simulation

In [8]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation 
model_voters = init_model_voters(N, M, K)

# Data Collection
data_voters = run!(model_voters, agent_step!, nsteps);

update_score_flow!(model_voters)

sort!(DataFrame(allagents(model_voters)), :score, rev = true)

MethodError: MethodError: no method matching init_model_voters(::Int64, ::Int64, ::Int64)
Closest candidates are:
  init_model_voters(::Int64, ::Int64, ::Int64, !Matched::Any) at In[6]:4

### Candidate Simulation

In [9]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation 
model_candidates = init_model_candidates(model_voters, M, K)

# Data Collection
#data_candidates = run!(model_candidates, agent_step!, nsteps);

update_score_candidates!(model_candidates)

sort!(DataFrame(allagents(model_candidates)), :final_score, rev = true)

UndefVarError: UndefVarError: model_voters not defined

### Visualization 

In [52]:
V = model_voters.space.graph
C = model_candidates.space.graph
V = ElectoralPhysics.LightGraphs_to_NetworkX(V);
V = nx.convert_node_labels_to_integers(V, first_label=1);
nx.write_graphml(V, "random_graph_flow_all.graphml")
C = ElectoralPhysics.LightGraphs_to_NetworkX(C);
C = nx.convert_node_labels_to_integers(C, first_label=1);
nx.write_graphml(C, "random_graph_flow_candidates.graphml")

In [ ]:
#nw.visualize(C)

In [45]:
#LightGraphs.weights(model_candidates.space.graph)[rand(1:M), :]

5-element SparseArrays.SparseVector{Float64,Int64} with 3 stored entries:
  [1]  =  0.333333
  [4]  =  0.333333
  [5]  =  0.333333

### @PietroMonticone2 Twitter Graph 

In [28]:
𝒫 = nx.read_graphml("@PietroMonticone2.graphml")
nx.convert_node_labels_to_integers(𝒫, first_label=1)
nx.write_edgelist(𝒫, "graph.edgelist")
𝒫 = loadgraph("graph.edgelist", GraphIO.EdgeList.EdgeListFormat())
𝒫 = SimpleWeightedDiGraph(reverse(𝒫))

{472504, 1075454} directed simple Int64 graph with Float64 weights

In [29]:
typeof(𝒫)

SimpleWeightedDiGraph{Int64,Float64}

In [30]:
indegrees = indegree(𝒫);

In [72]:
M = length([elem for elem in indegrees if elem != 0])

1157

In [34]:
length(indegrees)

472504

In [77]:
# Voter Model Initialization
function init_model_voters_twitter(𝒫::SimpleWeightedDiGraph{Int64,Float64}, indegrees::Array{Int64})
       
    #properties = @dict(N, M, K)
    
    # Create Directed Weighted Graph
    space = GraphSpace(𝒫) 
    model_voters_twitter = ABM(Voter, space) #; properties = properties)
    
    N = nv(𝒫)
    pos = 0
    
    for indegree in indegrees
        pos += 1
        score = 1.0
        final_score = 0
        if indegree == 0
            candidacy = false
            add_agent!(pos, model_voters_twitter, candidacy, score, final_score, 1)
        else
            candidacy = true
            add_agent!(pos, model_voters_twitter, candidacy, score, final_score, 1)
        end
    end
    return model_voters_twitter
end

# Candidate Model Initialization

function init_model_candidates_twitter(model_voters_twitter::AgentBasedModel{Voter,GraphSpace{SimpleWeightedDiGraph{Int64,Float64}},typeof(fastest),Nothing}, M::Int64)
    properties = @dict(M)
    
    candidates = []
    i=0
    for row in eachrow(sort!(DataFrame(allagents(model_voters_twitter)), :score, rev = true))
        if i == M
            break
        end
        append!(candidates, row[1])
        i+=1
    end
    
    # Induce Subgraph
    graph, vertex_map = induced_subgraph(model_voters_twitter.space.graph, [candidate for candidate in candidates])
    space = GraphSpace(graph)
    model_candidates_twitter = ABM(Voter, space; properties = properties)
    
    for candidate in allagents(model_voters_twitter)
        if candidate.candidacy == true
            add_agent!(candidate.pos, model_candidates_twitter, candidate.candidacy, candidate.score, candidate.final_score, candidate.community)
        end
    end
    
    for agent in allagents(model_candidates_twitter)
    agent.id = agent.pos
    end
    
    return model_candidates_twitter
end;

In [68]:
function twitter_vote!(agent, model)
    outneighbors = node_neighbors(agent, model; neighbor_type=:in)
    nvotes = length(outneighbors)
    for outneighbor in outneighbors
        agent_outneighbor = get_node_agents(outneighbor, model)[1]
        add_edge!(model.space.graph, agent.pos, agent_outneighbor.pos, 1/nvotes)
    end
end

function update_score_twitter_flow!(model)
    for agent in allagents(model)
        if agent.candidacy == true
            for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
                agent_inneighbor = get_node_agents(inneighbor, model)[1]
                if agent_inneighbor.candidacy == false
                    agent.score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
                end
            end
        end
    end
end

function update_score_candidates_twitter!(model)
    for agent in allagents(model)
        for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
            agent_inneighbor = get_node_agents(inneighbor, model)[1]
            agent.final_score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
        end
    end
end

function twitter_agent_step!(agent, model)
   twitter_vote!(agent, model) 
end;

### Twitter Voter Simulation

In [46]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation 
model_voters_twitter = init_model_voters_twitter(𝒫, indegrees)

data_voters = run!(model_voters_twitter, twitter_agent_step!, 1);

update_score_twitter_flow!(model_voters_twitter)

sort!(DataFrame(allagents(model_voters_twitter)), :score, rev = true)

,id,pos,candidacy,score,final_score,community
,Int64,Int64,Bool,Float64,Float64,Int64
1,557,557,1,4906.39,0.0,1
2,281,281,1,4904.69,0.0,1
3,730,730,1,4890.68,0.0,1
4,1129,1129,1,4855.79,0.0,1
5,41,41,1,4852.82,0.0,1
6,1133,1133,1,4794.64,0.0,1
7,514,514,1,4794.21,0.0,1
8,252,252,1,4760.96,0.0,1
9,57,57,1,4728.06,0.0,1


In [48]:
G = ElectoralPhysics.LightGraphs_to_NetworkX(model_voters_twitter.space.graph);
G = nx.convert_node_labels_to_integers(G, first_label=1);
nx.write_graphml(G, "twitter_election.graphml")

### Twitter Candidates Simulation

In [78]:
# Model Instantiation 
model_candidates_twitter = init_model_candidates_twitter(model_voters_twitter, M)

# Data Collection
#data_candidates = run!(model_candidates, agent_step!, nsteps);

update_score_candidates_twitter!(model_candidates_twitter)

sort!(DataFrame(allagents(model_candidates_twitter)), :final_score, rev = true)

,id,pos,candidacy,score,final_score,community
,Int64,Int64,Bool,Float64,Float64,Int64
1,105,105,1,3954.21,7984.2,1
2,255,255,1,3195.72,7615.76,1
3,758,758,1,3170.9,7200.88,1
4,658,658,1,3119.67,7149.65,1
5,522,522,1,2375.7,6795.74,1
6,845,845,1,2004.29,6424.33,1
7,740,740,1,2019.34,6049.32,1
8,638,638,1,1436.39,5828.63,1
9,871,871,1,1719.12,5749.1,1


In [79]:
G = ElectoralPhysics.LightGraphs_to_NetworkX(model_candidates_twitter.space.graph);
G = nx.convert_node_labels_to_integers(G, first_label=1);
nx.write_graphml(G, "twitter_election_candidates.graphml")

In [ ]:
nw.visualize(G)